This jupyter notebook will develp the SOP for updating the Monday.com studies board consistently.

In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import re
from datetime import datetime


In [2]:
input_dir = Path("/Users/hinashah/Documents/HEAL/MondayFolderUpdate_202406_CTN/")

In [3]:
def get_unique_values(df:pd.DataFrame, col_name:str='appl_id'):
    if col_name in df.columns:
        return df[ ~pd.isna(df[col_name])][col_name].drop_duplicates()
    return None

def get_na_count(df:pd.DataFrame, col_name:str='appl_id'):
    if col_name in df.columns:
        return len(df[pd.isna(df[col_name])])
    return -1

In [4]:
gt_file = pd.read_csv(input_dir/"study_lookup_table.csv", dtype=str)
gt_file.replace("0", np.NaN, inplace=True)

print(len(gt_file))
print(gt_file.columns)
print(gt_file.dtypes)
### QC the file:
for k in gt_file.columns:
    print(f"Number of distinct values in --{k}--: {len(get_unique_values(gt_file, k))}")
    print(f"---- NA count: {get_na_count(gt_file, k)}")
    ## Look for patterns?
    if 'appl' in k:
        d = gt_file[[ (not pd.isna(l)) and (not l.isdigit()) for l in gt_file[k] ]]
        print(f"Number of funky looking appl_ids: {len(d)}")
    elif k == 'study_hdp_id':
        d = gt_file[ [ (not pd.isna(l)) and (re.match(r'HDP[\d]+', l) is None) for l in gt_file[k]]]
        print(f" Number of funky looking HDPIDs: {len(d)}")

1476
Index(['appl_id', 'xstudy_id', 'study_most_recent_appl', 'study_hdp_id',
       'study_hdp_id_appl'],
      dtype='object')
appl_id                   object
xstudy_id                 object
study_most_recent_appl    object
study_hdp_id              object
study_hdp_id_appl         object
dtype: object
Number of distinct values in --appl_id--: 1467
---- NA count: 0
Number of funky looking appl_ids: 0
Number of distinct values in --xstudy_id--: 1198
---- NA count: 0
Number of distinct values in --study_most_recent_appl--: 1189
---- NA count: 0
Number of funky looking appl_ids: 0
Number of distinct values in --study_hdp_id--: 1160
---- NA count: 51
 Number of funky looking HDPIDs: 0
Number of distinct values in --study_hdp_id_appl--: 1151
---- NA count: 51
Number of funky looking appl_ids: 0


In [5]:
## Import Monday Board 
## TODO: change to read in all groups
board_files_list = list(input_dir.glob("HEAL_Studies_*.xlsx"))
print(board_files_list)
monday_board = pd.DataFrame()
for file_path in board_files_list:
    tmp_df = pd.read_excel(file_path, skiprows=4, dtype={"Most Recent Appl_ID":str, "Name":str}, skipfooter=1)
    group_name = ' '.join(file_path.name.split('_')[2:-1])
    if group_name == 'Studies never added to the Platform legacy':
        study_type = 'APPLIDONLY'
    elif group_name == 'HEAL Studies in the Platform':
        study_type = "HDP"
    elif group_name == "CTN Protocols":
        study_type == "CTN"
    else:
        study_type = "Unknown"
    tmp_df['study_type'] = [study_type]*len(tmp_df)
    monday_board = pd.concat([monday_board, tmp_df])
    print(len(monday_board))

# monday_board = pd.read_excel(input_dir/"HEAL_Studies_1719341536.xlsx", skiprows=4, dtype={"Most Recent Appl_ID":str}, skipfooter=1)
# print(monday_board["Name"].describe())
# print(monday_board.columns)

[PosixPath('/Users/hinashah/Documents/HEAL/MondayFolderUpdate_202406_CTN/HEAL_Studies_Studies_never_added_to_the_Platform_legacy_1719346227.xlsx'), PosixPath('/Users/hinashah/Documents/HEAL/MondayFolderUpdate_202406_CTN/HEAL_Studies_HEAL_Studies_in_the_Platform_1719346253.xlsx')]
38
1208


/var/folders/tb/c_qhpk1j2jj3knzfnw72yj080000gq/T/ipykernel_9064/1014237317.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  monday_board = pd.concat([monday_board, tmp_df])


In [6]:
### Steps for updating Monday board:

## From Study lookup table, get unique set of most_recent_appl, study_hdp_id, and study_hdp_id_appl
lookup_fields = gt_file[['study_hdp_id', 'study_most_recent_appl', 'study_hdp_id_appl']].copy(deep=True).drop_duplicates()
## Create a column "Name" or "Key" that will either have study_hdp_id OR most_recent_appl when study_hdp_id is empty
lookup_fields['key'] = [m if pd.isna(h) else h for (h, m) in lookup_fields[['study_hdp_id', 'study_most_recent_appl']].values ]

### A few checks:
## How many of the "keys" from Monday board are in lookup fields?
print(f"Number records from Monday already in lookup table: {len(monday_board[monday_board.Name.isin(lookup_fields.key)])}")
## How many of the keys from MOnday board are not there in looup fields
mondayboard_missingin_looup = monday_board[~monday_board.Name.isin(lookup_fields.key)]
print(f"Number records from Monday that are not in lookup table: {len(mondayboard_missingin_looup)}")
## How many of the keys from lookup fields are not there in Monday??
lookup_missingin_mondayboard = lookup_fields[~lookup_fields.key.isin(monday_board.Name)]
print(f"Number records from l0ookup that are not in Monday: {len(lookup_missingin_mondayboard)}")


Number records from Monday already in lookup table: 1198
Number records from Monday that are not in lookup table: 10
Number records from l0ookup that are not in Monday: 0


In [7]:
mondayboard_missingin_looup

,Name,Most Recent Appl_ID,HDP appl_ID,Project #,Archived,HEAL-Related,Research Focus,Research Program,Title,Contact PI,...,Repo per PI,Repo per Platform,Platform Reg Time,CEDAR Form %,Repo Mapping,repo_22_2,repo_22_3,Creation Log,"link to ""Get the Data"" Engagement Board",study_type
411,HDP00475,9916202,9916202.0,3UG1DA015831-17S6,NaN,NaN,New Strategies to Prevent and Treat Opioid Add...,Prevention of Progression to Moderate or Sever...,The National Drug Abuse Treatment Clinical Tri...,"WEISS, ROGER D.",...,NaN,NaN,NaT,0.0,NIDA Data Share,NaN,NaN,"Kathy Jooss Nov 29, 2021 7:59 AM",NaN,HDP
430,HDP00501,9905155,9905155.0,3UG1DA015815-18S6,NaN,NaN,New Strategies to Prevent and Treat Opioid Add...,Prevention of Progression to Moderate or Sever...,Western States Node of the National Drug Abuse...,"SORENSEN, JAMES L",...,NaN,NaN,NaT,0.0,NIDA Data Share,NaN,NaN,"Kathy Jooss Nov 29, 2021 7:59 AM",NaN,HDP
497,HDP00614,9914612,9914612.0,3UG1DA013035-17S9,NaN,NaN,New Strategies to Prevent and Treat Opioid Add...,Prevention of Progression to Moderate or Sever...,NIDA Clinical Trials Network: Greater New York...,"ROTROSEN, JOHN P",...,NaN,NaN,2022-12-21,84.6,NIDA Data Share,NaN,NaN,"Kathy Jooss Nov 29, 2021 7:59 AM",NaN,HDP
499,HDP00617,9905154,9905154.0,3UG1DA015815-18S5,NaN,NaN,New Strategies to Prevent and Treat Opioid Add...,Prevention of Progression to Moderate or Sever...,Western States Node of the National Drug Abuse...,"SORENSEN, JAMES L",...,NaN,NaN,NaT,0.0,NIDA Data Share,NaN,NaN,"Kathy Jooss Nov 29, 2021 7:59 AM",NaN,HDP
553,HDP00695,9951611,9951611.0,3UG1DA015815-17S6,NaN,NaN,New Strategies to Prevent and Treat Opioid Add...,Prevention of Progression to Moderate or Sever...,Western States Node of the National Drug Abuse...,"SORENSEN, JAMES L",...,NaN,NaN,NaT,0.0,NIDA Data Share,NaN,NaN,"Kathy Jooss Nov 29, 2021 7:59 AM",NaN,HDP
574,HDP00724,9951612,9951612.0,3UG1DA015815-17S7,NaN,NaN,New Strategies to Prevent and Treat Opioid Add...,Prevention of Progression to Moderate or Sever...,Western States Node of the National Drug Abuse...,"SORENSEN, JAMES L",...,NaN,NaN,NaT,0.0,NIDA Data Share,NaN,NaN,"Kathy Jooss Nov 29, 2021 7:59 AM",NaN,HDP
580,HDP00734,10140552,10140552.0,3UG1DA040316-06S3,NaN,NaN,New Strategies to Prevent and Treat Opioid Add...,Optimizing Care for People with Opioid Use Dis...,Suicide Prediction and Prevention for People a...,"BART, GAVIN",...,NaN,NIDA Data Share,NaT,0.0,Figshare,NaN,NaN,"Kathy Jooss Nov 29, 2021 7:59 AM",NaN,HDP
611,HDP00897,10573954,10573954.0,3UG1DA015831-21S3,NaN,NaN,Cross-Cutting Research,Training the Next Generation of Researchers in...,National Institute on Drug Abuse Clinical Tria...,"D'ONOFRIO, GAIL",...,NaN,NIDA Data Share,NaT,0.0,NIDA Data Share,NaN,NaN,"Kathy Jooss May 4, 2023 11:25 AM",NaN,HDP
797,HDP01275,10583135,10583135.0,3UG1DA015815-21S1,NaN,NaN,New Strategies to Prevent and Treat Opioid Add...,Prevention of Progression to Moderate or Sever...,Western States Node of the National Drug Abuse...,"KORTHUIS, PHILIP TODD",...,NaN,NaN,NaT,0.0,NIDA Data Share,NaN,NaN,"Kathy Jooss Sep 5, 2023 5:51 PM",NaN,HDP
852,HDP01251,9895173,9895173.0,3UG1DA013035-18S2,NaN,Y,NaN,NaN,NIDA Clinical Trials Network: Greater New York...,"ROTROSEN, JOHN P",...,NaN,NaN,NaT,0.0,NaN,NaN,NaN,"Hina Shah Jun 19, 2024 1:22 PM",NaN,HDP


In [8]:
# Get rest of the tables
convert_dict = {'appl_id':str}

awards_df = pd.read_csv(input_dir/"awards.csv", low_memory=False, dtype=convert_dict)
awards_df = awards_df.dropna(how='all')
print(f"Awards table has: {len(awards_df)} entries, with {len(get_unique_values(awards_df))} appl_ids")
reporter_df = pd.read_csv(input_dir/"reporter.csv", low_memory=False, dtype=convert_dict)
reporter_df = reporter_df.dropna(how='all')
print(f"Reporter table has: {len(reporter_df)} entries, with {len(get_unique_values(reporter_df))} appl_ids")
progress_tracker_df = pd.read_csv(input_dir/"progress_tracker.csv", low_memory=False, dtype=convert_dict)
print(f"Platform generated table has: {len(progress_tracker_df)} entries, with {len(get_unique_values(progress_tracker_df))} appl_ids")
print(f"Platform table has {len(get_unique_values(progress_tracker_df))} unique HDP IDs")
repo_maping_df = pd.read_csv(input_dir/"repo_mapping.csv", low_memory=False, dtype=convert_dict)
print(f"Repo mapping table has: {len(repo_maping_df)} entrie, with {len(get_unique_values(repo_maping_df))} appl_ids")
pi_emails_df = pd.read_csv(input_dir/"pi_emails.csv", low_memory=False, dtype=convert_dict)
print(f"Repo mapping table has: {len(pi_emails_df)} entrie, with {len(get_unique_values(pi_emails_df))} appl_ids")
resnet_df = pd.read_csv(input_dir/"research_networks.csv", low_memory=False, dtype=convert_dict)
print(f"Research Network table has: {len(resnet_df)} entrie, with {len(get_unique_values(resnet_df))} appl_ids")


Awards table has: 1618 entries, with 1618 appl_ids
Reporter table has: 1617 entries, with 1617 appl_ids
Platform generated table has: 1319 entries, with 1290 appl_ids
Platform table has 1290 unique HDP IDs
Repo mapping table has: 1323 entrie, with 1323 appl_ids
Repo mapping table has: 1059 entrie, with 1059 appl_ids
Research Network table has: 1669 entrie, with 1669 appl_ids


In [9]:
## Manipulate emails to carry forward emails from a previous appl_id to the most recent one according to the lookup table and email table
appl_ids = gt_file[['appl_id', 'study_most_recent_appl']].drop_duplicates()
print(len(appl_ids))
appl_ids_emails = pd.merge(appl_ids, pi_emails_df, how='left', on='appl_id')

most_recent_emails = appl_ids_emails[ ~pd.isna(appl_ids_emails.pi_email)][['study_most_recent_appl', 'pi_email']].drop_duplicates()
most_recent_emails.rename(columns={'pi_email':'pi_email_latest'}, inplace=True)
print(most_recent_emails)
email_counts = most_recent_emails.groupby('study_most_recent_appl').size()
appl_ids_counts = appl_ids_emails.groupby('study_most_recent_appl').size()

print(email_counts.describe())
appl_ids_emails['email_count'] = [email_counts[k] if k in email_counts else 0 for k in appl_ids_emails['study_most_recent_appl']]
appl_ids_emails['applid_count'] = [appl_ids_counts[k] if k in appl_ids_counts else 0 for k in appl_ids_emails['study_most_recent_appl']]
appl_ids_emails['pi_email'].fillna('', inplace=True)
appl_ids_emails['keep'] = [1 if (c==0 or (c==1 and len(e)>0) or (c>1 and a==m)) else 0 for (c,a,m,e) in appl_ids_emails[['email_count', 'appl_id', 'study_most_recent_appl', 'pi_email' ]].values]
print(len(appl_ids_emails))

pi_emails_df_updated = appl_ids_emails[appl_ids_emails['keep']==1][['study_most_recent_appl', 'pi_email']].drop_duplicates()
pi_emails_df_updated['pi_email'] = [k.strip() for k in pi_emails_df_updated['pi_email']]
print(pi_emails_df_updated)

## Get Monday board emails, and fill in any that are different from mysql..
pi_emails_df_updated_monday = pd.merge(pi_emails_df_updated, monday_board[['Most Recent Appl_ID', 'Contact Email']].drop_duplicates(), how='left', left_on='study_most_recent_appl', right_on='Most Recent Appl_ID').drop(columns='Most Recent Appl_ID')
pi_emails_df_updated_monday['Contact Email'].replace('-', '', inplace=True)
pi_emails_df_updated_monday['Contact Email'].fillna('', inplace=True)
pi_emails_df_updated_monday['pi_email_updated'] = [me if (len(e)==0 and len(me) > 1) else e for (e,me) in pi_emails_df_updated_monday[['pi_email', 'Contact Email']].values]
print(pi_emails_df_updated_monday)
pi_emails_df_updated_monday.to_csv(input_dir/"email_updates.csv", index=False)
appl_ids_emails.to_csv(input_dir/"email_counts.csv", index=False)

pi_emails_df_updated = pi_emails_df_updated_monday[['study_most_recent_appl', 'pi_email_updated']].rename(columns={'pi_email_updated':'pi_email'})

1467
     study_most_recent_appl                   pi_email_latest
1                   9755001                 kwatkins@rand.org
2                   9850412                   damico@rand.org
4                  10478911                 LYNN.DEBAR@KP.ORG
8                  10468778          cheville.andrea@mayo.edu
11                 10054792                   xcao11@jhmi.edu
...                     ...                               ...
1454               10167785               bahmedani@yahoo.com
1455               10331849                  tbrocki1@JHU.EDU
1457               10197811                  kzivin@UMICH.EDU
1458               10197809            Gregory.E.Simon@kp.org
1460                9823898  d-mencihella@md.northwestern.edu

[875 rows x 2 columns]
count    874.000000
mean       1.001144
std        0.033826
min        1.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        2.000000
dtype: float64
1467
     study_most_recent_appl                  pi

/var/folders/tb/c_qhpk1j2jj3knzfnw72yj080000gq/T/ipykernel_9064/1495752057.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  appl_ids_emails['pi_email'].fillna('', inplace=True)
/var/folders/tb/c_qhpk1j2jj3knzfnw72yj080000gq/T/ipykernel_9064/1495752057.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values 

In [10]:
## Collect fields from report/awards tables that are required by Monday Board
rename_dict = {'proj_num':'Project #', 
               'proj_title':'Title',
                'rfa':'Research Focus',
                'res_prg':'Research Program',
                'ctc_pi_nm':'Contact PI',
                'pi_email':'Contact Email',
                'adm_ic':'Administering IC',
                'prg_ofc':'NIH PO',
                'org_nm': 'Institution(s)',
                'pi':'PI(s)',
                'org_cy':'City',
                'org_st':'State',
                'act_code':'Activity Code',
                'awd_ty':'Award Type',
                'fisc_yr':'Award Year',
                'tot_fund':'Total Funded',
                'proj_abs':'Summary',
                'fund_mech': 'SBIR/STTR',
                'dai_res':'DAI Import Status',
                'proj_strt_date':'Project Start',
                'proj_end_date':'Project End',
                'proj_url':'Reporter Link',
                'res_net':'Network',
                'repo_22_1':'Repo Mapping',
                'repo_22_2':'repo_22_2',
                'repo_22_3':'repo_22_3',
                'time_of_registration':'Platform Reg Time',
                'overall_percent_complete':'CEDAR Form %',
                'repository_name' : 'Repo per Platform',
                'archived':'Archived',
                'heal_funded':'HEAL-Related'
                }

def create_mysql_subset(in_df:pd.DataFrame, extra_fields = ['appl_id']):
    subset = in_df[[k for k in rename_dict.keys() if k in in_df.columns] + extra_fields].copy(deep=True)
    subset.rename(columns={k:v for k,v in rename_dict.items() if k in in_df.columns}, inplace=True)
    return subset
    
mysql_fields_reporter = create_mysql_subset(awards_df)
mysql_fields_awards = create_mysql_subset(reporter_df)
myql_fields_repomapping = create_mysql_subset(repo_maping_df)
mysql_fields_platform = create_mysql_subset(progress_tracker_df, extra_fields=['hdp_id'])
mysql_fields_piemails = create_mysql_subset(pi_emails_df_updated, extra_fields=['study_most_recent_appl'])
mysql_fields_resnet = create_mysql_subset(resnet_df)

In [11]:
print(len(lookup_fields))
data_merge_1 = pd.merge(lookup_fields, mysql_fields_reporter, how='left', left_on='study_most_recent_appl', right_on='appl_id').drop(columns='appl_id')
print(len(data_merge_1))
data_merge_2 = pd.merge(data_merge_1, mysql_fields_awards, how='left', left_on='study_most_recent_appl', right_on='appl_id').drop(columns='appl_id')
print(len(data_merge_2))
data_merge_1 = pd.merge(data_merge_2, myql_fields_repomapping, how='left', left_on='study_most_recent_appl', right_on='appl_id').drop(columns='appl_id')
print(len(data_merge_1))
data_merge_2 = pd.merge(data_merge_1, mysql_fields_platform, how='left', left_on='study_hdp_id', right_on='hdp_id')
print(len(data_merge_2))
data_merge_1 = pd.merge(data_merge_2, mysql_fields_resnet, how='left', left_on='study_most_recent_appl', right_on='appl_id').drop(columns='appl_id')
print(len(data_merge_1))
combined_data_ph1 = pd.merge(data_merge_1, mysql_fields_piemails, how='left', on='study_most_recent_appl')
print(len(combined_data_ph1))
print(len(combined_data_ph1.drop_duplicates()))


1198
1198
1198
1198
1198
1198
1198
1198


In [12]:
## Find out which columns have NA values, and investigate for incompletemess?
print("Number of empty values for each of the fields gathered:")
for k in rename_dict.values():
    print(f"{k} : {get_na_count(combined_data_ph1, k)}")

Number of empty values for each of the fields gathered:
Project # : 0
Title : 0
Research Focus : 13
Research Program : 77
Contact PI : 0
Contact Email : 0
Administering IC : 0
NIH PO : 58
Institution(s) : 0
PI(s) : 0
City : 5
State : 7
Activity Code : 0
Award Type : 0
Award Year : 0
Total Funded : 0
Summary : 8
SBIR/STTR : 0
DAI Import Status : 708
Project Start : 0
Project End : 0
Reporter Link : 0
Network : 777
Repo Mapping : 731
repo_22_2 : 1090
repo_22_3 : 1190
Platform Reg Time : 864
CEDAR Form % : 38
Repo per Platform : 1056
Archived : 38
HEAL-Related : 21


In [18]:
## Add CTN  data
ctn_data = progress_tracker_df[[k.startswith('CTN') for k in progress_tracker_df['project_num']]]
ctn_data['project_title'].replace('0', '', inplace=True)

print(len(ctn_data))
rename_dict = {'project_num':'Project #', 
               'project_title':'Title',
                # 'rfa':'Research Focus',
                # 'res_prg':'Research Program',
                # 'investigators_name':'Contact PI',
                # 'pi_email':'Contact Email',
                # 'adm_ic':'Administering IC',
                # 'prg_ofc':'NIH PO',
                # 'org_nm': 'Institution(s)',
                'investigators_name':'PI(s)',
                # 'org_cy':'City',
                # 'org_st':'State',
                # 'act_code':'Activity Code',
                'award_type':'Award Type',
                'year_awarded':'Award Year',
                'award_amount':'Total Funded',
                'study_name':'Summary',
                # 'fund_mech': 'SBIR/STTR',
                # 'dai_res':'DAI Import Status',
                # 'proj_strt_date':'Project Start',
                'proj_end_date':'Project End',
                'nih_reporter_link':'Reporter Link',
                # 'res_net':'Network',
                # 'repo_22_1':'Repo Mapping',
                # 'repo_22_2':'repo_22_2',
                # 'repo_22_3':'repo_22_3',
                'time_of_registration':'Platform Reg Time',
                'overall_percent_complete':'CEDAR Form %',
                'repository_name' : 'Repo per Platform',
                'archived':'Archived',
                # 'heal_funded':'HEAL-Related'
                }
ctn_fields_platform = create_mysql_subset(ctn_data, extra_fields=['hdp_id'])
## Edit pi name
ctn_fields_platform['PI(s)'].fillna('', inplace=True)
ctn_fields_platform['PI(s)'] = [ k.translate(str.maketrans(',', ';', "[]\'")) for k in  ctn_fields_platform['PI(s)']]

ctn_fields_platform['key'] = ctn_fields_platform['hdp_id']
ctn_fields_platform['study_hdp_id'] = ctn_fields_platform['hdp_id']
ctn_fields_platform['Network'] = ['CTN']*len(ctn_fields_platform)
## Combine the data to the other data set
all_data = pd.concat([combined_data_ph1, ctn_fields_platform])
all_data


40


/var/folders/tb/c_qhpk1j2jj3knzfnw72yj080000gq/T/ipykernel_9064/493364525.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ctn_data['project_title'].replace('0', '', inplace=True)
/var/folders/tb/c_qhpk1j2jj3knzfnw72yj080000gq/T/ipykernel_9064/493364525.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ctn_data['project_title'].replace('0

,study_hdp_id,study_most_recent_appl,study_hdp_id_appl,key,Research Focus,Research Program,DAI Import Status,HEAL-Related,Project #,Title,...,Repo Mapping,repo_22_2,repo_22_3,Platform Reg Time,CEDAR Form %,Repo per Platform,Archived,hdp_id,Network,Contact Email
0,HDP00632,9860408,9860408,HDP00632,New Strategies to Prevent and Treat Opioid Add...,Preventing Opioid Use Disorder,NO,Y,3U01AA021691-08S1,National Consortium on Alcohol and Neurodevelo...,...,NIMH Data Archive,Vivli,NaN,NaN,0.0,NaN,live,HDP00632,NaN,
1,HDP00696,9755001,9755001,HDP00696,New Strategies to Prevent and Treat Opioid Add...,Optimizing Care for People with Opioid Use Dis...,NO,Y,3R34AA025480-02S1,Implementing Medication-Assisted Therapy for S...,...,Vivli,NaN,NaN,NaN,0.0,NaN,live,HDP00696,NaN,kwatkins@rand.org
2,HDP00509,9850412,9850412,HDP00509,New Strategies to Prevent and Treat Opioid Add...,Preventing Opioid Use Disorder,NO,Y,3R01AA025848-03S1,AOD Use Trajectories from Age 10 to 24: Multi-...,...,NIMH Data Archive,Vivli,NaN,NaN,0.0,NaN,live,HDP00509,NaN,damico@rand.org
3,HDP00242,10478911,9869480,HDP00242,Clinical Research in Pain Management,Pain Management Effectiveness Research Network,NaN,Y,5UH3AG067493-04,Tailored Non-Pharmacotherapy Services for Chro...,...,NaN,NaN,NaN,2023-02-14T10:30:05-08:00,80.8,NaN,live,HDP00242,Pain ERN,LYNN.DEBAR@KP.ORG
4,HDP00391,10468778,9876435,HDP00391,Clinical Research in Pain Management,Pragmatic and Implementation Studies for the M...,NaN,Y,5UH3AG067593-04,Non-pharmacological Options in postoperative H...,...,NaN,NaN,NaN,2024-02-08T05:59:13-08:00,23.1,ICPSR,live,HDP00391,PRISM,cheville.andrea@mayo.edu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1314,HDP01320,NaN,NaN,HDP01320,NaN,NaN,NaN,NaN,CTN-0110,,...,NaN,NaN,NaN,NaN,0.0,NaN,live,HDP01320,CTN,NaN
1315,HDP01321,NaN,NaN,HDP01321,NaN,NaN,NaN,NaN,CTN-0096,,...,NaN,NaN,NaN,NaN,0.0,NaN,live,HDP01321,CTN,NaN
1316,HDP01322,NaN,NaN,HDP01322,NaN,NaN,NaN,NaN,CTN-0084-A-2,,...,NaN,NaN,NaN,NaN,0.0,NaN,live,HDP01322,CTN,NaN
1317,HDP01323,NaN,NaN,HDP01323,NaN,NaN,NaN,NaN,CTN-0094,,...,NaN,NaN,NaN,NaN,0.0,NaN,live,HDP01323,CTN,NaN


In [19]:
combined_data = all_data.copy(deep=True)

In [20]:
## Create a column named "Location"dd
from datetime import datetime
combined_data['study_type'] = [ 'CTN' if m.startswith('CTN') else ('APPLIDONLY' if pd.isna(k) else 'HDP') for (m,k) in combined_data[['Project #', 'study_hdp_id_appl']].values]

combined_data['City'] = combined_data[['City']].fillna('')
combined_data['State'] = combined_data[['State']].fillna('')
combined_data['Location'] = [c+","+s for (c,s) in combined_data[['City', "State"]].values]

combined_data['Project Start'] = pd.to_datetime(combined_data['Project Start'], format='%Y-%m-%d', errors='coerce').dt.date
combined_data['Project End'] = pd.to_datetime(combined_data['Project End'], format='%Y-%m-%d', errors='coerce').dt.date
combined_data['Platform Reg Time'] = pd.to_datetime(combined_data['Platform Reg Time'], utc=True).dt.date


combined_data['Archived'] = [a if a=='archived' else '' for a in combined_data['Archived']]
combined_data['HEAL-Related'] = ['Y' if ((p is not 'CTN' ) and (pd.isna(a))) else '' for a in combined_data['HEAL-Related']]
combined_data['SBIR/STTR'] = [t if t=='SBIR/STTR' else '' for t in combined_data['SBIR/STTR']]

print(combined_data.study_type.value_counts())

## Rename a few of the other columns:
combined_data.rename(columns={'study_most_recent_appl':'Most Recent Appl_ID', 'study_hdp_id_appl':'HDP appl_ID'}, inplace=True)
combined_data.drop(columns=['study_hdp_id', 'hdp_id'], inplace=True)

combined_data.columns


study_type
HDP           1160
CTN             40
APPLIDONLY      38
Name: count, dtype: int64


Index(['Most Recent Appl_ID', 'HDP appl_ID', 'key', 'Research Focus',
       'Research Program', 'DAI Import Status', 'HEAL-Related', 'Project #',
       'Title', 'Contact PI', 'Administering IC', 'NIH PO', 'Institution(s)',
       'PI(s)', 'City', 'State', 'Activity Code', 'Award Type', 'Award Year',
       'Total Funded', 'Summary', 'SBIR/STTR', 'Project Start', 'Project End',
       'Reporter Link', 'Repo Mapping', 'repo_22_2', 'repo_22_3',
       'Platform Reg Time', 'CEDAR Form %', 'Repo per Platform', 'Archived',
       'Network', 'Contact Email', 'Location', 'study_type'],
      dtype='object')

In [21]:
#TODO: Find what's in Monday.com board, but not in mysql extract
# Mark these entries for deletion, and these would have to be deleted manually on Monday.com

In [22]:
combined_data.index.name = 'index'
combined_data.to_excel(input_dir/"MondayBoard_Update.xlsx")

print("Making sure uniqueness of key values")
key_counts = combined_data.groupby('key').size()
key_counts.describe()

Making sure uniqueness of key values


count    1238.0
mean        1.0
std         0.0
min         1.0
25%         1.0
50%         1.0
75%         1.0
max         1.0
dtype: float64

In [16]:
# combined_data_subset = combined_data[ combined_data['Most Recent Appl_ID'].isin(monday_board['Most Recent Appl_ID']) | combined_data['key'].isin(monday_board['Name'])].drop_duplicates()
# combined_data_subset = combined_data_subset[ ~pd.isna(combined_data_subset['Most Recent Appl_ID']) ]
# print(len(combined_data_subset))
# combined_data_subset.to_excel(input_dir/"MondayBoard_Update_Step2.xlsx", index=False)

In [17]:
# print(len(monday_board))
# missing_monday = combined_data_subset[~combined_data_subset.key.isin(monday_board.Name)]
# print(len(missing_monday))
# print(missing_monday.key)

# ## Did all the most_recent_appl_ids make it into the subset
# print(f"Number of most recent applids on Monday: {len(monday_board['Most Recent Appl_ID'].drop_duplicates())}, in subset: {len(combined_data_subset['Most Recent Appl_ID'].drop_duplicates())}")
# print(monday_board[ ~monday_board['Most Recent Appl_ID'].isin(combined_data_subset['Most Recent Appl_ID']) ]['Most Recent Appl_ID'])

# ## What is in Monday that is not in this combined dataset?
# missing_frommonday_inextract = monday_board[~monday_board.Name.isin(combined_data_subset.key)]
# print(len(missing_frommonday_inextract))



*******************
*******************
DEBUG CODE BELOW
*******************
*******************

In [18]:
combined_data[combined_data['key']=='HDP00889'].drop_duplicates()

,Most Recent Appl_ID,HDP appl_ID,key,Research Focus,Research Program,DAI Import Status,HEAL-Related,Project #,Title,Contact PI,...,repo_22_2,repo_22_3,Platform Reg Time,CEDAR Form %,Repo per Platform,Archived,Network,Contact Email,Location,study_type
index,,,,,,,,,,,,,,,,,,,,,
660,10593312,10601172,HDP00889,NaN,NaN,NaN,Y,3R24DA055306-02S1,Wake Forest IMPOWR Dissemination Education and...,"ADAMS, MEREDITH C. B.",...,NaN,NaN,2022-07-26,5.8,NaN,,NaN,meradams@wakehealth.edu,"WINSTON-SALEM,NC",HDP


In [19]:
comparison_cols = ['Most Recent Appl_ID', 'HDP appl_ID', 'Contact Email', 'Network']

comparison_df = pd.merge(monday_board[['Name'] +comparison_cols ], combined_data[['key'] + comparison_cols], left_on = 'Name', right_on='key').drop_duplicates()
comparison_df.to_csv(input_dir/"comparison.csv", index=False)
comparison_df.to_excel(input_dir/"comparison.xlsx", index=False)

In [20]:
comparison_df.columns

Index(['Name', 'Most Recent Appl_ID_x', 'HDP appl_ID_x', 'Contact Email_x',
       'Network_x', 'key', 'Most Recent Appl_ID_y', 'HDP appl_ID_y',
       'Contact Email_y', 'Network_y'],
      dtype='object')

In [21]:
pi_emails_df_updated[ pi_emails_df_updated.study_most_recent_appl=='9901704']

,study_most_recent_appl,pi_email
881,9901704,jhambm@upmc.edu


In [22]:
resnet_df[resnet_df.appl_id=='9908734']

,appl_id,res_net,res_net_override_flag
1486,9908734,NaN,0


In [23]:
combined_data.to_excel(input_dir/"test.xlsx", index=False)

In [24]:
expected_study_groups = pd.read_csv(input_dir/"STEP1_MondayUpdate.csv", low_memory=False)
study_groups_to_keep = expected_study_groups[expected_study_groups.keep_count==1].most_recent_study_group.drop_duplicates().values

study_groups = monday_board['TEMP: most_recent_study_group']
missing_studygroups = [k for k in study_groups_to_keep if k not in study_groups.values]
print(len(missing_studygroups))
missing_studygroups

FileNotFoundError: [Errno 2] No such file or directory: '/Users/hinashah/Documents/HEAL/MondayFolderUpdate_202406/STEP1_MondayUpdate.csv'

In [ ]:
combined_data.columns

Index(['study_hdp_id', 'study_most_recent_appl', 'study_hdp_id_appl', 'key',
       'Research Focus', 'TEMP_Res_Prog', 'Data Mgmt', 'DAI Import Status',
       'Network', 'Is Heal Funded or Relevant', 'Project #', 'Title',
       'Conctact PI', 'Administering IC', 'NIH PO', 'Institution(s)', 'PI(s)',
       'City', 'State', 'Activity Code', 'Award Type', 'Total Funded',
       'Summary', 'Project Start', 'Project End', 'Reporter Link',
       'Repo Mapping', 'repo_22_2', 'repo_22_3', 'Platform Reg Time',
       'CEDAR Form %', 'Repo per Platform', 'Archived on Platform?', 'hdp_id',
       'Contact Email', 'Location'],
      dtype='object')